In [ ]:
import os
import pandas as pd
import re

# 设置目标文件夹路径
directory = r"D:\yyx\course\Summer_2024\Internship\image_generation\ImageGeneration_forMedicalApplications\ImageGeneration_forMedicalApplications\Dataset\dataset_George\generated_new"

# 初始化列表存储数据
data = []

# 遍历目录中的所有文件
for filename in os.listdir(directory):
    if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # 获取完整路径
        image_path = os.path.join(directory, filename)
        
        # 删除文件名最后的 "__rating"
        original_image_path = re.sub(r'__(\d+)\.', '.', image_path)
        
        # 使用正则表达式匹配文件名最后的单个数字
        match = re.search(r'__(\d+)\.', filename)
        if match:
            rating = int(match.group(1))  # 提取匹配到的数字并转换为整数
        else:
            rating = None  # 若无匹配数字，设为 None 或其他默认值
        
        # 添加到数据列表
        data.append([original_image_path, rating])

# 创建 DataFrame
df = pd.DataFrame(data, columns=["image_path", "rating"])

# 保存为 CSV 文件
csv_path = os.path.join(directory, "image_ratings.csv")
df.to_csv(csv_path, index=False, encoding='utf-8')

print(f"CSV 文件已生成：{csv_path}")

In [ ]:
import os
import pandas as pd
import re

# 设置目标文件夹路径
directory = r"D:\yyx\course\Summer_2024\Internship\image_generation\ImageGeneration_forMedicalApplications\ImageGeneration_forMedicalApplications\Dataset\dataset_George\generated_new"
ratings_csv_path = os.path.join(directory, "image_ratings.csv")
predictions_csv_path = r"D:\yyx\course\Summer_2024\Internship\image_generation\ImageGeneration_forMedicalApplications\ImageGeneration_forMedicalApplications\Dataset\dataset_George\predictions_without_gt.csv"

# 读取两个 CSV 文件
ratings_df = pd.read_csv(ratings_csv_path)
predictions_df = pd.read_csv(predictions_csv_path)

# 仅保留文件名用于匹配
ratings_df['Filename'] = ratings_df['image_path'].apply(lambda x: os.path.basename(x))
predictions_df['Filename'] = predictions_df['Image Path'].apply(lambda x: os.path.basename(x))

# 合并两个 DataFrame，基于文件名匹配
merged_df = ratings_df.merge(predictions_df[['Filename', 'Pred Angle1', 'Pred Angle2', 'Image Path']], on='Filename', how='inner')

# 更新 image_path 为 predictions CSV 的路径
merged_df['image_path'] = merged_df['Image Path']
merged_df.drop(columns=['Filename', 'Image Path'], inplace=True)

# 保存合并后的 CSV 文件
output_csv_path = os.path.join(directory, "image_ratings_with_predictions.csv")
merged_df.to_csv(output_csv_path, index=False, encoding='utf-8')

print(f"CSV 文件已生成：{output_csv_path}")




In [ ]:
def classify_na(angle):
    if angle >= 105:
        return "This high-resolution X-ray provides an exceptional view of the dog's hip joint structure, showing excellent hip conformation. The Norberg angle above 105 degrees reflects a stable and well-formed hip with optimal bone alignment."
    elif 100 <= angle < 105:
        return "The X-ray reveals the canine hip with sharp bone margins and excellent contrast, demonstrating good joint conformation. The Norberg angle of 101-105 degrees suggests strong acetabular coverage and well-aligned femoral heads."
    elif 95 <= angle < 100:
        return "This clear and well-structured X-ray image of the dog's hip joints highlights a fair hip condition. With a Norberg angle of 96-100 degrees, the joint structure appears near-normal, with minimal laxity."
    elif 90 <= angle < 95:
        return "The X-ray scan provides a detailed and high-contrast visualization of the pelvic bones, suggesting a borderline hip condition. The Norberg angle of 91-95 degrees indicates slight incongruency in the femoral head and acetabulum."
    elif 85 <= angle < 90:
        return "The high-resolution X-ray displays the canine hip bones with well-defined edges, showing mild hip dysplasia. The Norberg angle of 85-90 degrees suggests reduced joint congruity and early degenerative changes."
    elif 80 <= angle < 85:
        return "The high-resolution X-ray displays the canine hip bones with well-defined edges, showing moderate hip dysplasia. The Norberg angle of 81-85 degrees suggests reduced joint congruity and early degenerative changes."
    elif angle < 80:
        return "This X-ray clearly visualizes the dog's hip structure, revealing severe hip dysplasia with a Norberg angle below 80 degrees, indicating significant joint instability and poor acetabular coverage."
    else:
        return "Unknown"

# 计算 Prompt 列，使用两个角度的平均值
merged_df['Prompt'] = merged_df[['Pred Angle1', 'Pred Angle2']].mean(axis=1).apply(classify_na)
merged_df.drop(columns=['Pred Angle1', 'Pred Angle2'], inplace=True)


In [ ]:
output_csv_path = os.path.join(directory, "image_ratings_with_predictions.csv")
merged_df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [2]:
import os
import pandas as pd
import re
path = "/home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv"
merged_df = pd.read_csv(path)
merged_df['modality'] = "CXR"
merged_df['expert_level'] = 1
merged_df.to_csv(path, index=False, encoding='utf-8')


In [3]:
import pandas as pd

# 读取 CSV 文件
path = "/home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv"
df = pd.read_csv(path)

# 修改列名
df.rename(columns={"Prompt": "prompt"}, inplace=True)

# 保存修改后的 CSV 文件
df.to_csv(path, index=False, encoding='utf-8')

print(f"CSV 文件已更新，并将 'Prompt' 列改为 'prompt'，路径: {path}")


CSV 文件已更新，并将 'Prompt' 列改为 'prompt'，路径: /home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv


In [5]:
import pandas as pd
import os

# 读取 CSV 文件
path = "/home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv"
df = pd.read_csv(path)

# 修改 image_path，使其指向 /home/featurize/data/Images/generated_new/
df["image_path"] = df["image_path"].apply(lambda x: os.path.join("/home/featurize/data/Images/generated_new", os.path.basename(x)))

# 保存修改后的 CSV 文件
df.to_csv(path, index=False, encoding='utf-8')

print(f"CSV 文件已更新，'image_path' 已修改为 '/home/featurize/data/Images/generated_new/' 目录，路径: {path}")


CSV 文件已更新，'image_path' 已修改为 '/home/featurize/data/Images/generated_new/' 目录，路径: /home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv


In [9]:
import os
import pandas as pd

resized_dir = "/home/featurize/data/RLHF/resized"
csv_output_path = "/home/featurize/data/RLHF/norberg_angles.csv"

if os.path.exists(csv_output_path):
    df = pd.read_csv(csv_output_path)
    df["image_path"] = df["image_path"].astype(str)  # 确保数据为字符串格式
    df["image_name"] = df["image_path"].apply(lambda x: os.path.basename(x.replace("\\", "/")))  # 解析图片名，兼容不同路径格式
    print(df["image_name"][0])
    df["image_path"] = df["image_name"].apply(lambda x: os.path.join(resized_dir, x))
    df.to_csv(csv_output_path, index=False)
    print(f"Updated image paths in {csv_output_path}!")
else:
    print(f"CSV file {csv_output_path} not found.")




236945.jpg
Updated image paths in /home/featurize/data/RLHF/norberg_angles.csv!


In [10]:
import pandas as pd
import numpy as np

# 读取 CSV 文件
path = "/home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv"
df = pd.read_csv(path)

# 查找包含 NaN 或 非有限值（inf）的行
df = df[df["rating"].apply(lambda x: np.isfinite(x))]


df = df[["image_path", "rating", "modality", "expert_level", "prompt"]]

# 保存修改后的 CSV 文件
df.to_csv(path, index=False, encoding='utf-8')

print(f"CSV 文件已更新，列顺序调整为 ['image_path', 'rating', 'modality', 'expert_level', 'prompt']，路径: {path}")


CSV 文件已更新，列顺序调整为 ['image_path', 'rating', 'modality', 'expert_level', 'prompt']，路径: /home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv


In [11]:
df["rating"] = df["rating"].astype(int)
df.to_csv(path, index=False, encoding='utf-8')

In [19]:
import pandas as pd

# 读取 CSV 文件
path = "/home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv"
df = pd.read_csv(path)

# 修改 'modality' 列的所有值
df['modality'] = "Chest X-Ray"

# 保存修改后的文件
output_path = path
df.to_csv(output_path, index=False)

print(f"修改后的文件已保存至: {output_path}")


修改后的文件已保存至: /home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv


In [20]:
import pandas as pd

path = "/home/featurize/work/Img_gen/Image_generation_with_labels/Minim_RLHF.csv"
df = pd.read_csv(path)

# 查找 NaN
print(df.isna().sum())



image_path      0
rating          0
modality        0
expert_level    0
prompt          0
dtype: int64


In [23]:
df['rating'] = df['rating'].astype(int)  # 确保都是整数
df = df[df['rating'].isin([1, 2, 3, 4, 5])]





In [12]:
import os
import pandas as pd

# 定义路径
csv_input_path = "/home/featurize/data/RLHF/norberg_angles.csv"

# 读取 CSV 并修改列
if os.path.exists(csv_input_path):
    df = pd.read_csv(csv_input_path)
    
    # 删除 image_name 列
    df = df.drop(columns=["image_name"], errors="ignore")
    
    # 保存修改后的 CSV
    df.to_csv(csv_input_path, index=False)
    print(f"Updated CSV saved as {csv_input_path}!")
else:
    print(f"CSV file {csv_input_path} not found.")


Updated CSV saved as /home/featurize/data/RLHF/norberg_angles.csv!


In [11]:
import os
import pandas as pd

# 定义路径
csv_input_path = "/home/featurize/data/RLHF/norberg_angles.csv"
csv_output_path = "/home/featurize/data/RLHF/norberg_angles_modified.csv"

# 读取 CSV 并修改列
if os.path.exists(csv_input_path):
    df = pd.read_csv(csv_input_path)
    
    # 删除 rating、expert_level 和 image_name 列
    df = df.drop(columns=["rating", "expert_level", "image_name"], errors="ignore")
    
    # 重命名列
    df = df.rename(columns={"image_path": "path", "prompt": "text"})
    
    # 修改 modality 列所有内容为 "CXR"
    df["modality"] = "CXR"
    
    # 保存为新的 CSV
    df.to_csv(csv_output_path, index=False)
    print(f"Modified CSV saved as {csv_output_path}!")
else:
    print(f"CSV file {csv_input_path} not found.")



Modified CSV saved as /home/featurize/data/RLHF/norberg_angles_modified.csv!


In [13]:
import pandas as pd
import os
import re

# 原始 CSV 文件路径
file_path = "/home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/image_ratings_with_predictions.csv"

# 读取 CSV 文件
df = pd.read_csv(file_path)

# 目标路径前缀
old_path_prefix = r"D:/yyx/course/Summer_2024/Internship/image_generation/ImageGeneration_forMedicalApplications/ImageGeneration_forMedicalApplications/Dataset/dataset_George/Dog_hip_RLHF_gren/outputs/XR/"
new_path_prefix = "/home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/Chest X-Ray/"

# 修改路径和文件名的函数
def modify_image_path(image_path):
    if old_path_prefix in image_path:
        # 提取原始文件名
        file_name = os.path.basename(image_path)
        # 提取数字和 policy 相关信息
        match = re.search(r'dog_hip_(\d+)_selector_([\d.]+)_policy_([\d.]+).png', file_name)
        if match:
            num, selector, policy = match.groups()
            # 生成新的文件名格式
            new_file_name = f"Chest X-Ray_{num}_selector_{selector}_policy_{policy}.png"
            return os.path.join(new_path_prefix, new_file_name).replace("\\", "/")
    return image_path  # 如果路径不匹配，保持原样

# 应用修改
df["image_path"] = df["image_path"].apply(modify_image_path)

# 保存修改后的 CSV
modified_file_path = file_path.replace("image_ratings_with_predictions.csv", "image_ratings_with_predictions_modified.csv")
df.to_csv(modified_file_path, index=False)

print(f"修改后的 CSV 已保存至: {modified_file_path}")


修改后的 CSV 已保存至: /home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/image_ratings_with_predictions_modified.csv


In [14]:

df["modality"] = "Chest X-Ray"
df["expert_level"] = 1
df.rename(columns={"Prompt": "prompt"}, inplace=True)
# 保存修改后的 CSV
modified_file_path = file_path.replace("image_ratings_with_predictions.csv", "image_ratings_with_predictions_modified.csv")
df.to_csv(modified_file_path, index=False)

print(f"修改后的 CSV 已保存至: {modified_file_path}")

修改后的 CSV 已保存至: /home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/image_ratings_with_predictions_modified.csv


In [15]:
import pandas as pd

# 原始 CSV 文件路径
file_path = "/home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/image_ratings_stage1.csv"

# 读取 CSV 文件
df = pd.read_csv(file_path)

# 旧的路径前缀和新的路径前缀
old_path_prefix = "/home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/Chest X-Ray/"
new_path_prefix = "/home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/Chest X-Ray_stage1/"

# 修改路径的函数
def modify_image_path(image_path):
    if image_path.startswith(old_path_prefix):
        return image_path.replace(old_path_prefix, new_path_prefix, 1)  # 只替换前缀部分
    return image_path  # 保持原样

# 应用修改
df["image_path"] = df["image_path"].apply(modify_image_path)

# 保存修改后的 CSV
modified_file_path = file_path.replace("image_ratings_stage1.csv", "image_ratings_stage1_modified.csv")
df.to_csv(modified_file_path, index=False)

print(f"修改后的 CSV 已保存至: {modified_file_path}")


修改后的 CSV 已保存至: /home/featurize/work/Img_gen/Image_generation_with_labels/MINIM/MINIM_RLHF/outputs/image_ratings_stage1_modified.csv
